In [45]:
from pyspark.sql.types import StructType, StructField, FloatType, LongType, StringType, IntegerType
from pyspark.shell import spark

In [46]:
attrs = []
f = open('./GDELT-EVENTS-ATTRIBUTES.txt')

for line in f:
    tokens = line.split(',')
    if tokens[1].strip() == "INTEGER":
        attrs.append(StructField(tokens[0].strip(), IntegerType(), True))
    elif tokens[1].strip() == "STRING":
        attrs.append(StructField(tokens[0].strip(), StringType(), True))
    elif tokens[1].strip() == "FLOAT":
        attrs.append(StructField(tokens[0].strip(), FloatType(), True))
    
schema = StructType(attrs)
schema

StructType(List(StructField(GLOBALEVENTID,IntegerType,true),StructField(SQLDATE,IntegerType,true),StructField(MonthYear,IntegerType,true),StructField(Year,IntegerType,true),StructField(FractionDate,FloatType,true),StructField(Actor1Code,StringType,true),StructField(Actor1Name,StringType,true),StructField(Actor1CountryCode,StringType,true),StructField(Actor1KnownGroupCode,StringType,true),StructField(Actor1EthnicCode,StringType,true),StructField(Actor1Religion1Code,StringType,true),StructField(Actor1Religion2Code,StringType,true),StructField(Actor1Type1Code,StringType,true),StructField(Actor1Type2Code,StringType,true),StructField(Actor1Type3Code,StringType,true),StructField(Actor2Code,StringType,true),StructField(Actor2Name,StringType,true),StructField(Actor2CountryCode,StringType,true),StructField(Actor2KnownGroupCode,StringType,true),StructField(Actor2EthnicCode,StringType,true),StructField(Actor2Religion1Code,StringType,true),StructField(Actor2Religion2Code,StringType,true),StructFie

In [54]:
df = spark.read.format('CSV').option('sep', '\t').schema(schema).load('inputs/gdelt/2016*')

In [55]:
df.take(1)

[Row(GLOBALEVENTID=597122373, SQLDATE=20151110, MonthYear=201511, Year=2015, FractionDate=2015.8492431640625, Actor1Code=None, Actor1Name=None, Actor1CountryCode=None, Actor1KnownGroupCode=None, Actor1EthnicCode=None, Actor1Religion1Code=None, Actor1Religion2Code=None, Actor1Type1Code=None, Actor1Type2Code=None, Actor1Type3Code=None, Actor2Code='BUS', Actor2Name='BOSS', Actor2CountryCode=None, Actor2KnownGroupCode=None, Actor2EthnicCode=None, Actor2Religion1Code=None, Actor2Religion2Code=None, Actor2Type1Code='BUS', Actor2Type2Code=None, Actor2Type3Code=None, IsRootEvent=1, EventCode='042', EventBaseCode='042', EventRootCode='04', QuadClass=1, GoldsteinScale=1.899999976158142, NumMentions=6, NumSources=1, NumArticles=6, AvgTone=0.8064516186714172, Actor1Geo_Type=0, Actor1Geo_FullName=None, Actor1Geo_CountryCode=None, Actor1Geo_ADM1Code=None, Actor1Geo_Lat=None, Actor1Geo_Long=None, Actor1Geo_FeatureID=None, Actor2Geo_Type=4, Actor2Geo_FullName='Sunshine Coast, Queensland, Australia', A

In [56]:
df.count()

73270827

In [57]:
df_2015 = spark.read.format('CSV').option('sep', '\t').schema(schema).load('inputs/gdelt/2015*')

In [58]:
df_2015.take(1)

[Row(GLOBALEVENTID=478037761, SQLDATE=20051025, MonthYear=200510, Year=2005, FractionDate=2005.8082275390625, Actor1Code='CVL', Actor1Name='POPULATION', Actor1CountryCode=None, Actor1KnownGroupCode=None, Actor1EthnicCode=None, Actor1Religion1Code=None, Actor1Religion2Code=None, Actor1Type1Code='CVL', Actor1Type2Code=None, Actor1Type3Code=None, Actor2Code=None, Actor2Name=None, Actor2CountryCode=None, Actor2KnownGroupCode=None, Actor2EthnicCode=None, Actor2Religion1Code=None, Actor2Religion2Code=None, Actor2Type1Code=None, Actor2Type2Code=None, Actor2Type3Code=None, IsRootEvent=0, EventCode='110', EventBaseCode='110', EventRootCode='11', QuadClass=3, GoldsteinScale=-2.0, NumMentions=25, NumSources=4, NumArticles=25, AvgTone=-2.1553008556365967, Actor1Geo_Type=4, Actor1Geo_FullName='Tehran, Tehran, Iran', Actor1Geo_CountryCode='IR', Actor1Geo_ADM1Code='IR26', Actor1Geo_Lat=35.75, Actor1Geo_Long=51.514801025390625, Actor1Geo_FeatureID='10074674', Actor2Geo_Type=0, Actor2Geo_FullName=None,

In [59]:
df_2015.count()

66370819